# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os

### Step 1: Scope the Project and Gather Data

#### Scope [TODO]
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data [TODO]
Describe the data sets you're using. Where did it come from? What type of information is included? 

**I94 Immigration Data:** This data comes from the [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html), and it contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries). You can find a detailed description at *./I94_SAS_Labels_Descriptions.SAS*

In [2]:
! ls -lh ../../data/18-83510-I94-Data-2016/

total 6.0G
-rw-r--r-- 1 root root 451M May 31  2018 i94_apr16_sub.sas7bdat
-rw-r--r-- 1 root root 597M May 31  2018 i94_aug16_sub.sas7bdat
-rw-r--r-- 1 root root 500M May 31  2018 i94_dec16_sub.sas7bdat
-rw-r--r-- 1 root root 374M May 31  2018 i94_feb16_sub.sas7bdat
-rw-r--r-- 1 root root 415M May 31  2018 i94_jan16_sub.sas7bdat
-rw-r--r-- 1 root root 620M May 31  2018 i94_jul16_sub.sas7bdat
-rw-r--r-- 1 root root 684M May 31  2018 i94_jun16_sub.sas7bdat
-rw-r--r-- 1 root root 459M May 31  2018 i94_mar16_sub.sas7bdat
-rw-r--r-- 1 root root 501M May 31  2018 i94_may16_sub.sas7bdat
-rw-r--r-- 1 root root 424M May 31  2018 i94_nov16_sub.sas7bdat
-rw-r--r-- 1 root root 531M May 31  2018 i94_oct16_sub.sas7bdat
-rw-r--r-- 1 root root 543M May 31  2018 i94_sep16_sub.sas7bdat


In [3]:
# fname0 = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# df0 = pd.read_sas(fname0, 'sas7bdat', encoding="ISO-8859-1")
fname0 = './immigration_data_sample.csv'
df0 = pd.read_csv(fname0)

In [4]:
df0.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

df_spark0 =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [6]:
#write to parquet
if not os.path.isdir("./sas_data"):
    df_spark0.write.parquet("sas_data")
df_spark0=spark.read.parquet("sas_data")

**World Temperature Data:** This dataset came from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

**Columns**

*dt:* date

*AverageTemperature:* Average temperature of the date

*AverageTemperatureUncertainty*: The 95% confidence interval around the average

*City:* City

*Country:* The country where the city belongs to

*Latitude:* Latitude of the city

*Longitude:* Longitude of the city

In [7]:
! ls -lh ../../data2

total 509M
-rw-r--r-- 1 1002 1003 509M Mar 30  2019 GlobalLandTemperaturesByCity.csv


In [8]:
fname1 = '../../data2/GlobalLandTemperaturesByCity.csv'
df1 = pd.read_csv(fname1)

In [9]:
df1.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


**U.S. City Demographic Data:** This data comes from [OpenSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/). This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000. This data comes from the US Census Bureau's 2015 American Community Survey. Data Schema is presented [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).

In [10]:
fname2 = './us-cities-demographics.csv'
df2 = pd.read_csv(fname2, delimiter=';')

In [11]:
df2.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


**Airport Code Table:** This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data). You can find field information in the above link.

In [12]:
fname3 = './airport-codes_csv.csv'
df3 = pd.read_csv(fname3)

In [13]:
df3.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [14]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [15]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [16]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.